In [1]:
import tensorflow as tf
import librosa
import numpy as np
from pydub import AudioSegment, effects
import noisereduce as nr

/opt/homebrew/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load in model

In [12]:
# Load first model
saved_model_path = './six_model.json'
saved_weights_path = './six_model_weights.h5'

# Load new, better model
# saved_model_path = './goodmodel.json'
# saved_weights_path = './goodmodel_weights.h5'

with open(saved_model_path , 'r') as json_file:
    json_savedModel = json_file.read()

model = tf.keras.models.model_from_json(json_savedModel)  # Assuming json_savedModel is defined
model.load_weights(saved_weights_path)
model.compile(loss='categorical_crossentropy', 
                optimizer='RMSProp', 
                metrics=['categorical_accuracy'])

Preprocessing

In [6]:
import tensorflow as tf
import librosa
import numpy as np
from pydub import AudioSegment, effects
import noisereduce as nr

total_length = 173056 # desired frame length for all of the audio samples.
def preprocess(file_path, frame_length = 2048, hop_length = 512):
    '''
    A process to an audio .wav file before execcuting a prediction.
      Arguments:
      - file_path - The system path to the audio file.
      - frame_length - Length of the frame over which to compute the speech features. default: 2048
      - hop_length - Number of samples to advance for each frame. default: 512

      Return:
        'X_3D' variable, containing a shape of: (batch, timesteps, feature) for a single file (batch = 1).
    ''' 
    # Fetch sample rate.
    _, sr = librosa.load(path = file_path, sr = None)
    # Load audio file
    rawsound = AudioSegment.from_file(file_path, duration = None) 
    # Normalize to 5 dBFS 
    normalizedsound = effects.normalize(rawsound, headroom = 5.0) 
    # Transform the normalized audio to np.array of samples.
    normal_x = np.array(normalizedsound.get_array_of_samples(), dtype = 'float32')
# Trim silence from the beginning and the end.
    xt, index = librosa.effects.trim(normal_x, top_db=30)
    try:
        padded_x = np.pad(xt, (0, total_length-len(xt)), 'constant')
    except:
        print("error")
        print("file:", file)
        return None
    # normal_x = np.array(normalizedsound.get_array_of_samples(), dtype = 'float32') 
    # Noise reduction                  
    final_x = nr.reduce_noise(padded_x, sr=sr)
        
        
    f1 = librosa.feature.rms(y=final_x, frame_length=frame_length, hop_length=hop_length, center=True, pad_mode='reflect').T # Energy - Root Mean Square
    f2 = librosa.feature.zero_crossing_rate(y=final_x, frame_length=frame_length, hop_length=hop_length,center=True).T # ZCR
    f3 = librosa.feature.mfcc(y=final_x, sr=sr, S=None, n_mfcc=13, hop_length = hop_length).T # MFCC   
    X = np.concatenate((f2, f1, f3), axis = 1)
    
    X_3D = np.expand_dims(X, axis=0)
    
    return X_3D

# file_path = './test_files/copy_OAF_fail_disgust.wav'  # Update with the path to your .wav file
# file_path = './own_recordings/rec12.wav'
file_path = '../final_test/steven_happy_2.wav'

preprocessed_file = preprocess(file_path)

In [13]:
predictions = model.predict(preprocessed_file)
np.set_printoptions(suppress=True)
print("---------")
print("File:", file_path)
print(predictions)

# Assuming your model outputs a softmax distribution over emotions
emotions = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful']  
print(np.argmax(predictions))
predicted_emotion = emotions[np.argmax(predictions)]
print(f"Predicted emotion: {predicted_emotion}")

1/1 [==============================] - 0s 327ms/step
---------
File: ../final_test/luke_long_1.wav
[[0.0011203  0.00100184 0.02389973 0.05518602 0.06875975 0.8500323 ]]
5
Predicted emotion: fearful


### Workflow for processing multiple files at once

In [14]:
import os

preprocessed_files = []
preprocessed_files_names = []
directory = '../final_test'
for file in os.listdir(directory):
    if '.DS_Store' in file:
      continue
    file_path = os.path.join(directory, file)
    processed_file = preprocess(file_path)
    if processed_file is not None:
        preprocessed_files.append(processed_file)
        preprocessed_files_names.append(file)

for i, f in enumerate(preprocessed_files):
    emotions = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful']  

    predictions = model.predict(f)
    rounded_predictions = np.round(predictions[0], 3)
    # predictions_list = rounded_predictions.tolist()
    # print(predictions_list)
    np.set_printoptions(suppress=True)
    print("---------")
    print("File:", preprocessed_files_names[i])
    print(' '.join(f'{col:<8}' for col in emotions))
    print(' '.join(f'{str(val):<8}' for val in rounded_predictions))

    # Assuming your model outputs a softmax distribution over emotions
 
    print(np.argmax(predictions))
    predicted_emotion = emotions[np.argmax(predictions)]
    print(f"Predicted emotion: {predicted_emotion}")


error
file: luke_long_1.wav
1/1 [==============================] - 0s 45ms/step
---------
File: luke_sad_2.wav
neutral  calm     happy    sad      angry    fearful 
0.001    0.005    0.016    0.706    0.006    0.267   
3
Predicted emotion: sad
1/1 [==============================] - 0s 66ms/step
---------
File: luke_happy_1.wav
neutral  calm     happy    sad      angry    fearful 
0.001    0.0      0.005    0.987    0.0      0.007   
3
Predicted emotion: sad
1/1 [==============================] - 0s 22ms/step
---------
File: luke_sad_1.wav
neutral  calm     happy    sad      angry    fearful 
0.0      0.001    0.009    0.245    0.015    0.73    
5
Predicted emotion: fearful
1/1 [==============================] - 0s 22ms/step
---------
File: luke_happy_2.wav
neutral  calm     happy    sad      angry    fearful 
0.003    0.0      0.0      0.997    0.0      0.0     
3
Predicted emotion: sad
1/1 [==============================] - 0s 23ms/step
---------
File: steven_neutral_1.wav
neutral  ca

### Analysis of longer audio files by breaking them down

In [104]:
import librosa
import numpy as np
import matplotlib.pyplot as plt

# Variables
FILE_PATH = './final_test/luke_long_1.wav'
SAMPLE_RATE = 24414  # Same as RATE in the original script
CHUNK_DURATION = 1  # Duration of chunks to analyze (in seconds)

# Function to split audio file into 1-second chunks and process each chunk
def process_audio_file(file_path):
    total_predictions = []  # List to store predictions for each chunk
    
    # Load audio file
    audio, sr = librosa.load(file_path, sr=SAMPLE_RATE)
    
    # Calculate the number of chunks
    num_chunks = int(np.floor(len(audio) / sr / CHUNK_DURATION))
    
    for i in range(num_chunks):
        # Extract the chunk
        start_sample = i * sr * CHUNK_DURATION
        end_sample = start_sample + sr * CHUNK_DURATION
        chunk = audio[start_sample:end_sample]
        
        # Save chunk to temporary WAV file or directly pass the audio array to your preprocessing function if possible
        chunk_file_path = 'temp_chunk.wav'
        librosa.output.write_wav(chunk_file_path, chunk, sr)
        
        # Preprocess the chunk
        x = preprocess(chunk_file_path)  # Ensure your preprocess function is adapted for handling the chunk
        
        # Model's prediction => an 8 emotion probabilities array
        predictions = model.predict(x, use_multiprocessing=True)
        pred_list = list(predictions)
        pred_np = np.squeeze(np.array(pred_list).tolist(), axis=0)  # Simplify the predictions list
        total_predictions.append(pred_np)
        
        # Optionally, visualize the prediction for each chunk
        # emo_list should be defined as your list of emotions
        fig = plt.figure(figsize=(10, 2))
        plt.bar(emo_list, pred_np, color='darkturquoise')
        plt.ylabel("Probability (%)")
        plt.show()
        
        # Print the predominant emotion for the chunk
        max_emo = np.argmax(predictions)
        print('max emotion for chunk:', emotions.get(max_emo, -1))
        print(100 * '-')
    
    # After processing all chunks, you can aggregate the predictions if needed
    # For example, calculate the mean prediction across all chunks
    mean_predictions = np.mean(np.array(total_predictions), axis=0)
    fig = plt.figure(figsize=(10, 5))
    plt.bar(emo_list, mean_predictions, color='indigo')
    plt.ylabel("Mean probability (%)")
    plt.title("Overall Emotion Distribution")
    plt.show()

# Call the function to process the audio file
process_audio_file(FILE_PATH)


AttributeError: No librosa attribute output